In [1]:
#HomeGuard Security System Simulator
#Author: [Pedro Biain]
#Description: A smart home monitoring system that processes sensor readings and triggers alerts for security, safety, and comfort issues.


import random
from datetime import datetime

# System configuration
HOME_MODES = ["HOME", "AWAY", "SLEEP"]
ALERT_SEVERITIES = ["LOW", "MEDIUM", "HIGH", "CRITICAL"]

# Current system state
current_mode = "AWAY"

def create_sensor(sensor_id, location, sensor_type, threshold=None):
   #Creates a sensor data structure.

    """Parameters:
    - sensor_id: Unique identifier for the sensor
    - location: Where the sensor is located (e.g., "Living Room")
    - sensor_type: Type of sensor ("motion", "temperature", "door", "smoke")
    - threshold: Optional threshold value for the sensor

    Returns:
    - A dictionary representing the sensor
    """
    sensor = {'id':sensor_id, 'location': location, 'type':sensor_type, 'threshold':threshold}

    return sensor

def create_alert(severity, message, sensor_id, timestamp):
    """
    Creates an alert data structure.

    Parameters:
    - severity: Alert severity level (LOW, MEDIUM, HIGH, CRITICAL)
    - message: Description of the alert
    - sensor_id: ID of the sensor that triggered the alert
    - timestamp: When the alert was triggered

    Returns:
    - A dictionary representing the alert
    """
    alert = {'severity':severity, 'message': message, 'sensor_id':sensor_id, 'timestamp':timestamp}
    return alert

# Initialize sensors for the Peterson home
sensors = [
    create_sensor(1,"living room", "motion"), #motion for the living room
    create_sensor(2, 'kitchen', 'temperature', threshold={'min': 35, 'max': 95}), #temperature for the kitchen with threshold
    create_sensor(3, 'entrance', 'door'), #door for the front door
    create_sensor(4, 'bedroom', 'smoke')  #smoke for the bedroom
]

print(f"Initialized {len(sensors)} sensors")
for i in sensors:
    print(f"  - {i['id']}: {i['location']} ({i['type']})")


def is_abnormal_reading(sensor, reading_value):
    """
    Checks if a sensor reading is abnormal based on sensor type and thresholds.

    Parameters:
    - sensor: Sensor dictionary
    - reading_value: The current reading from the sensor

    Returns:
    - True if the reading is abnormal, False otherwise
    """
    sensor_type = sensor["type"]

    # YOUR CODE HERE: Add if/else logic to check for abnormal readings
    if sensor_type == "temperature":
        # For temperature sensors: check if below 35°F or above 95°F
        return reading_value < 35 or reading_value > 95
    elif sensor_type == "motion":
        # For motion sensors: check if motion is detected (reading_value == True)
        return reading_value == True
    elif sensor_type == "door":
        # For door sensors: check if door is open (reading_value == "OPEN")
        return reading_value == "OPEN"
    elif sensor_type == "smoke":
        # For smoke sensors: check if smoke is detected (reading_value == "DETECTED")
        return reading_value == "DETECTED"
    return False # Default to normal if sensor type not handled or reading is normal


def should_trigger_security_alert(sensor, reading_value, system_mode):
    """
    Determines if a security alert should be triggered.

    Parameters:
    - sensor: Sensor dictionary
    - reading_value: The current reading from the sensor
    - system_mode: Current system mode (HOME, AWAY, SLEEP)

    Returns:
    - True if a security alert should be triggered, False otherwise
    """
    if is_abnormal_reading(sensor, reading_value):
        sensor_type = sensor["type"]
        # Trigger security alert for motion/door if in AWAY or SLEEP mode
        if sensor_type in ["motion", "door"] and system_mode in ["AWAY", "SLEEP"]:
            return True
        # Trigger security alert for smoke regardless of mode (usually critical)
        elif sensor_type == "smoke":
            return True
    return False

    # Test temperature check
test_sensor = create_sensor("TEMP_TEST", "Test Room", "temperature", threshold=35)
print(f"34°F is abnormal: {is_abnormal_reading(test_sensor, 34)}")  # Should be True
print(f"68°F is abnormal: {is_abnormal_reading(test_sensor, 68)}")  # Should be False

# Test security alert
motion_sensor = create_sensor("MOTION_TEST", "Test Room", "motion")
print(f"Motion in AWAY mode triggers alert: {should_trigger_security_alert(motion_sensor, True, 'AWAY')}")  # Should be True
print(f"Motion in HOME mode triggers alert: {should_trigger_security_alert(motion_sensor, True, 'HOME')}")  # Should be False

def generate_reading(sensor):
    """
    Generates a realistic reading for a sensor.

    Parameters:
    - sensor: Sensor dictionary

    Returns:
    - A realistic reading value based on sensor type
    """

    # YOUR CODE HERE: Generate appropriate readings based on sensor type
    # Use random values to simulate real sensor data
    # - Temperature: random value between 30-100°F
    # - Motion: random boolean (True/False)
    # - Door: random choice between "OPEN" and "CLOSED"
    # - Smoke: random choice between "CLEAR" and "DETECTED"

    sensor_type = sensor["type"]
    if sensor_type == "temperature":
        return random.randint(30, 100)
    elif sensor_type == "motion":
        return random.choice([True, False])
    elif sensor_type == "door":
        return random.choice(["OPEN", "CLOSED"])
    elif sensor_type == "smoke":
        return random.choice(["CLEAR", "DETECTED"])
    return None


def process_reading(sensor, reading_value, system_mode):

    """
    Processes a sensor reading and determines if alerts are needed.

    Parameters:
    - sensor: Sensor dictionary
    - reading_value: The reading from the sensor
    - system_mode: Current system mode

    Returns:
    - A list of alerts (empty if no alerts needed)
    """
    alerts = []
    timestamp = datetime.now().strftime("%H:%M:%S")

    # YOUR CODE HERE: Check for different types of alerts
    # 1. Check for security alerts (motion/door in AWAY mode)
    # 2. Check for safety alerts (temperature extremes, smoke)
    # 3. Check for comfort notifications (temperature out of range in HOME mode)

    # Security alerts
    if should_trigger_security_alert(sensor, reading_value, system_mode):
        severity = "HIGH"
        message = f"Security alert: {sensor['type'].capitalize()} detected in {sensor['location']}!"
        if sensor['type'] == 'smoke':
            severity = "CRITICAL"
            message = f"CRITICAL Safety alert: Smoke detected in {sensor['location']}!"
        alerts.append(create_alert(severity, message, sensor['id'], timestamp))

    # Safety alerts (e.g., extreme temperature regardless of mode)
    elif sensor['type'] == "temperature":
        if is_abnormal_reading(sensor, reading_value):
            severity = "MEDIUM"
            message = f"Safety alert: Extreme temperature ({reading_value}°F) in {sensor['location']}."
            alerts.append(create_alert(severity, message, sensor['id'], timestamp))

    # Comfort notifications (only in HOME mode, if temperature is outside preferred range)
    if system_mode == "HOME" and sensor['type'] == "temperature":
        if reading_value < 65 or reading_value > 75: # Example comfort range
            severity = "LOW"
            message = f"Comfort notification: Temperature ({reading_value}°F) in {sensor['location']} is outside preferred range."
            alerts.append(create_alert(severity, message, sensor['id'], timestamp))

    return alerts

# Feel free to change the function signatures or add helper functions as needed, these are just some suggestions to make the code more friendly!
def trigger_alert(alert):
    """
    Displays an alert to the user.

    Parameters:
    - alert: Alert dictionary
    """
    severity_symbol = {
        "LOW": "ℹ️",
        "MEDIUM": "⚠️",
        "HIGH": "🚨",
        "CRITICAL": "🔥"
    }

    symbol = severity_symbol.get(alert["severity"], "⚠️")
    print(f"[ALERT!] {symbol} {alert['severity']}: {alert['message']}")

def log_event(message, timestamp=None):
    """
    Logs an event to the console.

    Parameters:
    - message: The message to log
    - timestamp: Optional timestamp (uses current time if not provided)
    """
    if timestamp is None:
        timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[LOG] [{timestamp}] {message}")

    # Test reading generation
test_sensor = sensors[0]  # Motion sensor
reading = generate_reading(test_sensor)
print(f"Generated reading for {test_sensor['location']}: {reading}")

# Test processing
alerts = process_reading(test_sensor, True, "AWAY")
if alerts:
    trigger_alert(alerts[0])

class Sensor:
    """
    Represents a sensor in the HomeGuard system.
    """

    def __init__(self, sensor_id, location, sensor_type, threshold=None):
        """
        Initializes a new sensor.

        Parameters:
        - sensor_id: Unique identifier for the sensor
        - location: Where the sensor is located
        - sensor_type: Type of sensor ("motion", "temperature", "door", "smoke")
        - threshold: Optional threshold value for the sensor
        """
        self.id = sensor_id
        self.location = location
        self.type = sensor_type
        self.threshold = threshold
        self.current_value = None # Initialize current_value to None


    def read(self):
        """
        Generates and stores a new reading for this sensor.

        Returns:
        - The reading value
        """
        # Use the global generate_reading function, converting self to a dict for compatibility
        sensor_dict = {
            "id": self.id,
            "location": self.location,
            "type": self.type,
            "threshold": self.threshold
        }
        self.current_value = generate_reading(sensor_dict)
        return self.current_value

    def isAbnormal(self):
        """
        Checks if the current reading is abnormal.

        Returns:
        - True if the reading is abnormal, False otherwise
        """
        if self.current_value is None:
            return False # Cannot be abnormal if no reading
        sensor_dict = {
            "id": self.id,
            "location": self.location,
            "type": self.type,
            "threshold": self.threshold
        }
        return is_abnormal_reading(sensor_dict, self.current_value)


    def reset(self):
        """
        Resets the sensor's current reading to None.
        """
        # YOUR CODE HERE: Set current_value to None
        self.current_value = None

    def __str__(self):
        """
        Returns a string representation of the sensor.
        """
        status = "No reading" if self.current_value is None else str(self.current_value)
        return f"{self.id} ({self.location}): {status}"

# Create sensor objects using the class
sensor_objects = [
    Sensor("MOTION_001", "Living Room", "motion"),
    Sensor("TEMP_001", "Kitchen", "temperature", threshold=35),
    Sensor("DOOR_001", "Front Door", "door"),
    Sensor("SMOKE_001", "Bedroom", "smoke")
]


def run_simulation(duration_minutes=5, system_mode="AWAY"):
    """
    Runs the HomeGuard security system simulation.

    Parameters:
    - duration_minutes: How long to run the simulation (default: 5 minutes)
    - system_mode: System mode (HOME, AWAY, SLEEP)
    """
    print("=" * 50)
    print("=== HomeGuard Security System ===")
    print("=" * 50)
    print(f"Mode: {system_mode}\n")

    # Use sensor objects instead of dictionaries
    sensors = [
        Sensor("MOTION_001", "Living Room", "motion"),
        Sensor("TEMP_001", "Kitchen", "temperature", threshold=35),
        Sensor("DOOR_001", "Front Door", "door"),
        Sensor("SMOKE_001", "Bedroom", "smoke")
    ]

    # Simulate time passing (each iteration = 1 minute)
    for minute in range(duration_minutes):
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"\nTime: {current_time}")

        # Read all sensors
        for sensor in sensors:
            reading = sensor.read()

            # Display the reading
            if sensor.type == "temperature":
                status = "Normal" if 65 <= reading <= 75 else "Abnormal"
                print(f"[READING] {sensor.location} Temperature: {reading}°F ({status})")
            elif sensor.type == "motion":
                status = "DETECTED" if reading else "No activity"
                print(f"[READING] {sensor.location} Motion: {status}")
            elif sensor.type == "door":
                print(f"[READING] {sensor.location}: {reading}")
            elif sensor.type == "smoke":
                print(f"[READING] {sensor.location} Smoke: {reading}")

            # Process the reading and trigger alerts if needed
            # Convert sensor object to dict format for process_reading function
            sensor_dict = {
                "id": sensor.id,
                "location": sensor.location,
                "type": sensor.type,
                "threshold": sensor.threshold
            }
            alerts = process_reading(sensor_dict, reading, system_mode)

            # Trigger all alerts
            for alert in alerts:
                trigger_alert(alert)
                if alert["severity"] in ["HIGH", "CRITICAL"]:
                    log_event("Sending notification to homeowner...")

        # Small delay to make output readable (optional)
        import time
        time.sleep(0.5)  # 0.5 second delay between iterations

    print("\n" + "=" * 50)
    print("Simulation complete!")
    print("=" * 50)

# Main execution
if __name__ == "__main__":
    # Run the simulation
    run_simulation(duration_minutes=3, system_mode="AWAY")

Initialized 4 sensors
  - 1: living room (motion)
  - 2: kitchen (temperature)
  - 3: entrance (door)
  - 4: bedroom (smoke)
34°F is abnormal: True
68°F is abnormal: False
Motion in AWAY mode triggers alert: True
Motion in HOME mode triggers alert: False
Generated reading for living room: False
[ALERT!] 🚨 HIGH: Security alert: Motion detected in living room!
=== HomeGuard Security System ===
Mode: AWAY


Time: 15:24:42
[READING] Living Room Motion: DETECTED
[ALERT!] 🚨 HIGH: Security alert: Motion detected in Living Room!
[LOG] [15:24:42] Sending notification to homeowner...
[READING] Kitchen Temperature: 99°F (Abnormal)
[ALERT!] ⚠️ MEDIUM: Safety alert: Extreme temperature (99°F) in Kitchen.
[READING] Front Door: CLOSED
[READING] Bedroom Smoke: DETECTED
[ALERT!] 🔥 CRITICAL: CRITICAL Safety alert: Smoke detected in Bedroom!
[LOG] [15:24:42] Sending notification to homeowner...

Time: 15:24:42
[READING] Living Room Motion: DETECTED
[ALERT!] 🚨 HIGH: Security alert: Motion detected in Livi